In [ ]:
import os
import langchain_chroma
import openai
from openai import OpenAI
import chromadb
import fitz  # PDF에서 텍스트 추출
from dotenv import load_dotenv
from fastapi import FastAPI, UploadFile, File
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import psycopg2
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from getpass import getpass

# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
POST_DB_HOST = os.getenv("POST_DB_HOST")
POST_DB_NAME = os.getenv("POST_DB_NAME")
POST_DB_USER = os.getenv("POST_DB_USER")
POST_DB_PASSWD = os.getenv("POST_DB_PASSWD")
POST_DB_PORT = os.getenv("POST_DB_PORT")
# PostgreSQL 연결 엔진 생성
post_engine = create_engine(f'postgresql://{POST_DB_USER}:{POST_DB_PASSWD}@{POST_DB_HOST}:{POST_DB_PORT}/{POST_DB_NAME}')
db = psycopg2.connect(host=POST_DB_HOST, dbname=POST_DB_NAME,user=POST_DB_USER,password=POST_DB_PASSWD,port=POST_DB_PORT)
cursor = db.cursor()
db.autocommit = False

# 데이터 불러오기
# 이력서 불러오기
path = "./data/빅데이터AI_이력서.pdf"
doc = fitz.open(path)
for page in doc:
    resume_text = page.get_text()
    print(resume_text)

📄 이력서 (Resume)
[기본 정보]
이름: 홍길동
연락처: 010-1234-5678
이메일: honggildong.ai@gmail.com
주소: 서울특별시 강남구 테헤란로 123
[학력]
고려대학교 컴퓨터학과 졸업 (2018.03 ~ 2024.02)
GPA: 3.85 / 4.5
관련 과목: 머신러닝, 데이터마이닝, 통계학, 빅데이터처리, 딥러닝 이론과 실습
[기술 스택]
Programming: Python, SQL, R
Frameworks/Libraries: Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy
Tools: Jupyter, Git, Docker, Tableau
DBMS: MySQL, MongoDB, Hadoop(HDFS), Spark
Cloud: Google Colab, AWS EC2 & S3 (기초 수준)
[프로젝트 경험]
1. 신문 기사 기반 감성 분석 모델 개발 (2023.03 ~ 2023.06)
자연어처리(NLP) 기반 감성 분류 모델 개발
KoNLPy와 Scikit-learn을 이용한 전처리 및 모델 학습
정확도 86% 달성
2. 머신러닝 기반 개인 맞춤형 영화 추천 시스템 (2023.09 ~ 2023.12)
📄 이력서 (Resume)
1

Content-based Filtering 및 Collaborative Filtering 기법 적용
Streamlit으로 웹 인터페이스 구현
kaggle 데이터셋 기반, Precision@10: 0.73
[자격증]
ADsP (데이터분석 준전문가) – 2023.08
SQLD (SQL 개발자) – 2024.01
📄 이력서 (Resume)
2



In [3]:
df = pd.read_csv("../data_backup/rec_data.csv")

In [ ]:

from langchain.vectorstores import ElasticVectorSearch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.storage import InMemoryStore

In [32]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [51]:
index_name = "jd_index"
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "elastic7276"),  # 👈 설치 시 발급된 비번
    verify_certs=False
)

/root/miniconda3/envs/apps/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [46]:
def index_documents_to_es(index_name, documents):
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name)

    actions = [
        {
            "_index": index_name,
            "_id": i,
            "_source": {
                "content": doc.page_content,
                **doc.metadata
            }
        }
        for i, doc in enumerate(documents)
    ]

    bulk(es, actions)

In [35]:
class ElasticsearchRetriever:
    def __init__(self, es, index_name, k=5):
        self.es = es
        self.index_name = index_name
        self.k = k

    def get_relevant_documents(self, query):
        response = self.es.search(
            index=self.index_name,
            query={
                "match": {
                    "content": {
                        "query": query,
                        "operator": "and"
                    }
                }
            },
            size=self.k
        )
        hits = response["hits"]["hits"]
        return [
            Document(
                page_content=hit["_source"]["content"],
                metadata={k: v for k, v in hit["_source"].items() if k != "content"}
            )
            for hit in hits
        ]

In [6]:
df

,Unnamed: 0,rec_idx,chunk,company_nm,recruit_kewdcdnm,company_place,career,education
0,0,46254462,[3500/중식제공]식품 쇼핑몰 MD 경력직 채용\n\n채용공고 상세,(주)다온미트,['온라인MD'],경기 하남시,경력(년수무관) · 정규직,학력무관
1,1,46552267,iOS 개발자 경력 모집\n\n채용공고 상세\nIOS 개발자 모집모집부문 및 상세내...,한국전자인증㈜,['Objective-C'],서울 서초구,1 ~ 10년 · 정규직,학력무관
2,2,46809948,인하우스 디자인 / 시각 제품 그래픽 산업 3D UX UI 웹\n\n채용공고 상세\...,(주)아트앤매니지먼트,"['크리에이터', '광고디자인', '그래픽디자인', '디지털디자인', '산업디자인']",서울 강남구 외,신입 · 경력 · 정규직,학력무관
3,3,46858808,제주 드림타워 복합리조트 [카지노&호텔] 대규모 신입 공개 채용 채용공고 상세\n ...,(주)엘티엔터테인먼트,"['출납', '사무직', '문서작성', '비품관리', '사무보조']",제주 제주시 외,신입 · 정규직 외,학력무관
4,4,47037243,경영지원팀-경영지원(신입-경력)사원 구인 공고\n\n채용공고 상세\n빠르게 성장하는...,엑사옵토닉스(주),"['경리', '총무', '경영지원', '회계']",경기 화성시,경력무관 · 정규직,학력무관
...,...,...,...,...,...,...,...,...
13848,13848,50283723,[동인광학] 부설연구소 전자 FW/HW 개발자 모집\n\n채용공고 상세\n[동인광학...,(주)동인광학,"['연구원', 'R&D']",경기 부천시 오정구,경력 3년↑ · 정규직,"대학(2,3년)↑"
13849,13849,50283724,생산직 직원 모집합니다\n\n채용공고 상세\n안녕하세요 \n당사는 내화채움재등을 제...,주식회사더원에프에스,"['생산관리', 'MES', '생산']",경기 양주시,경력무관 · 정규직,학력무관
13850,13850,50283728,광주 에이치원 호텔 프론트 데스크/업무 직원\n\n채용공고 상세\n광주 에이치원 호...,(주)에이치원호텔,"['호텔리어', '객실관리', '고객안내', '고객응대', '프론트']",광주 북구,경력무관 · 정규직,고졸↑
13851,13851,50283729,경북어울림)포항시발달장애인주간활동센터 제공인력 채용\n\n채용공고 상세\n경북어울림...,경북어울림사회적협동조합,"['사회복지사', '생활복지사', '장애인복지']",경북 포항시 북구,경력무관 · 정규직 외,학력무관


In [7]:
from langchain.schema import Document

documents = []
for _, row in df.iterrows():
    doc = Document(
        page_content=row["chunk"],
        metadata={
            "rec_idx": row["rec_idx"],
            "company_nm": row["company_nm"],
            "recruit_kewdcdnm": row["recruit_kewdcdnm"],
            "company_place": row["company_place"],
            "career": row["career"],
            "education": row["education"],
        }
    )
    documents.append(doc)


In [17]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema.document import Document
import os

# 1. OpenAI 임베딩 모델
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. Chroma에서 벡터 리트리버 로딩 (이미 구축된 벡터 DB)
vectordb = Chroma(
    persist_directory="./chroma_data",
    embedding_function=embedding_model,
    collection_name="chroma_test"
)
vector_retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# 3. BM25Retriever (chunk 리스트 그대로 사용)
# 👉 'chunk' 컬럼과 metadata로 Document 객체 생성
from langchain.schema import Document

bm25_docs = [
    Document(page_content=row["chunk"], metadata=row.drop("chunk").to_dict())
    for _, row in df.iterrows()
]
bm25_retriever = BM25Retriever.from_documents(bm25_docs)
bm25_retriever.k = 20

# 4. 하이브리드 리트리버 조합
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.2, 0.8]  # → 가중치 조정 가능 (BM25 강화하려면 [0.7, 0.3]처럼)
)


In [30]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 정의
template = """
당신은 인재매칭 AI 어시스턴트입니다. 사용자 이력서에 기반하여 채용공고를 추천해주세요.

그리고 사용자 정보는 다음과 같습니다:
- 지원 직무: 데이터 분석, AI 모델링, 감성 분석, 영화 추천 시스템 개발
- 관심 산업/도메인: IT, 빅데이터
- 기술/스킬: Python, SQL, R, Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy, Jupyter, Git, Docker, Tableau, MySQL, MongoDB, Hadoop(HDFS), Spark, Google Colab, AWS EC2 & S3, KoNLPy, Word2Vec, Streamlit
- 학력 및 경력: 고려대학교 컴퓨터학과 졸업, 신입
- 희망 지역: 서울특별시 강남구

- 반드시 이력서에 기반할 것.
- 채용공고 상세는 반드시 모두 출력할 것.
- 출력 시 채용공고의 양식을 사용할 것.
- 채용공고 추천 이유를 한줄로 설명할 것.         
- 같은 공고 번호가 중복될 경우 단 1개만 추천할 것.                  
- 채용공고 전체 내용을 기반하여 분석할 것. 제목만 보고 판단하지 말 것.
- page_content의 전체 텍스트를 기준으로 판단할 것.

### 출력 형식 예시
1.
- 직무명 : 
- 공고 번호 :                         
- 산업 분야 : 
- 필요 기술 : 
- 채용공고 상세 :
- 추천 사유 :

이 정보를 바탕으로 사용자 정보에 가장 적합한 채용 공고 20개를 추천해주세요.
출력 형식과 동일한 형식으로 출력해주세요.                                          

채용공고:
{context}

이력서:
{question}
"""

prompt = PromptTemplate.from_template(template)

# LLM 세팅
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

# 전체 체인 정의
rag_chain = (
     {
        "context": hybrid_retriever | (lambda docs: "\n\n".join(
            f"{doc.metadata.get('rec_idx')}\n{doc.page_content.strip()}"
            for doc in docs
        )),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [31]:
# 사용 예시
question = resume_text
answer = rag_chain.invoke(question)
print(answer)

1.
- 직무명 : 데이터 분석가
- 공고 번호 : 50226095
- 산업 분야 : IT개발·데이터
- 필요 기술 : 데이터분석, 머신러닝, 딥러닝, 빅데이터
- 채용공고 상세 : LG CNS에서 데이터 분석가를 포함한 다양한 IT 직무를 모집하며, 머신러닝, 딥러닝, 빅데이터 관련 업무를 수행할 인재를 찾고 있습니다.
- 추천 사유 : 사용자의 데이터 분석 및 AI 모델링 기술이 LG CNS의 데이터 분석가 직무와 잘 맞습니다.

2.
- 직무명 : 소프트웨어 개발자
- 공고 번호 : 50275838
- 산업 분야 : 소프트웨어 개발
- 필요 기술 : Python, SQL, RESTful API, Git
- 채용공고 상세 : 소프트웨어 개발자를 모집하며, Python과 SQL을 사용한 데이터 처리 및 RESTful API 설계 경험이 요구됩니다.
- 추천 사유 : 사용자의 Python 및 SQL 기술이 이 직무에 적합합니다.

3.
- 직무명 : 데이터 취득 관리자
- 공고 번호 : 50282644
- 산업 분야 : 데이터 관리
- 필요 기술 : Python, AWS, 데이터 전처리
- 채용공고 상세 : 데이터 취득 및 관리 업무를 담당하며, AWS를 통한 데이터 관리 및 Python을 사용한 데이터 전처리 경험이 필요합니다.
- 추천 사유 : 사용자의 AWS 및 Python 기술이 데이터 취득 관리 업무에 적합합니다.

4.
- 직무명 : 글로벌 세일즈 매니저
- 공고 번호 : 50281662
- 산업 분야 : 글로벌 세일즈
- 필요 기술 : 데이터 분석, 시장 조사
- 채용공고 상세 : 글로벌 세일즈 매니저로서 시장 조사 및 데이터 분석을 통해 전략을 개발하고, 해외 바이어와의 관계를 구축합니다.
- 추천 사유 : 사용자의 데이터 분석 능력이 글로벌 세일즈 전략 개발에 유용할 것입니다.

5.
- 직무명 : 웹 프론트엔드 개발자
- 공고 번호 : 49948486
- 산업 분야 : 웹 개발
- 필요 기술 : JavaScript, Node.js, RESTf

---

In [36]:
class HybridRetriever:
    def __init__(self, es_retriever, vector_retriever, alpha=0.5):
        self.es_retriever = es_retriever
        self.vector_retriever = vector_retriever
        self.alpha = alpha  # 0~1 벡터 중요도

    def get_relevant_documents(self, query):
        es_docs = self.es_retriever.get_relevant_documents(query)
        vec_docs = self.vector_retriever.get_relevant_documents(query)

        combined = es_docs[:int((1 - self.alpha) * 10)] + vec_docs[:int(self.alpha * 10)]
        # 중복 제거
        seen = set()
        final = []
        for doc in combined:
            key = doc.page_content
            if key not in seen:
                final.append(doc)
                seen.add(key)
        return final

In [37]:
documents

[Document(metadata={'rec_idx': 46254462, 'company_nm': '(주)다온미트', 'recruit_kewdcdnm': "['온라인MD']", 'company_place': '경기 하남시', 'career': '경력(년수무관) · 정규직', 'education': '학력무관'}, page_content='[3500/중식제공]식품 쇼핑몰 MD 경력직 채용\n\n채용공고 상세'),
 Document(metadata={'rec_idx': 46552267, 'company_nm': '한국전자인증㈜', 'recruit_kewdcdnm': "['Objective-C']", 'company_place': '서울 서초구', 'career': '1 ~ 10년 · 정규직', 'education': '학력무관'}, page_content='iOS 개발자 경력 모집\n\n채용공고 상세\nIOS 개발자 모집모집부문 및 상세내용공통 자격요건ㆍ학력 : 대졸 이상 (4년) IOS 개발자 모집AI Security 연구본부 0명지원자격ㆍ경력 : 경력 2년 이상 우대사항ㆍ컴퓨터/시스템공학근무조건ㆍ근무형태:정규직(수습기간)-6개월ㆍ근무일시:주 5일(월~금)ㆍ근무지역: 서울 서초구 서초동 전형절차서류전형- 코딩테스트-1차면접 -2차면접-영어면접(필요시)-임원면접- 최종합격접수기간 및 방법ㆍ접수기간:2023년 9월 11일 (월) 10시 ~ 2023년 10월 11일 (수) 24시ㆍ접수방법:사람인 입사지원ㆍ이력서양식:사람인 온라인 이력서유의사항ㆍ입사지원 서류에 허위사실이 발견될 경우, 채용 확정 이후라도 채용이 취소될 수 있습니다.'),
 Document(metadata={'rec_idx': 46809948, 'company_nm': '(주)아트앤매니지먼트', 'recruit_kewdcdnm': "['크리에이터', '광고디자인', '그래픽디자인', '디지털디자인', '산업디자인']", 'company_place': '서울 강남구 외', 'career': '신입 · 경력

In [52]:
index_documents_to_es(index_name, documents)

/root/miniconda3/envs/apps/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/miniconda3/envs/apps/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/miniconda3/envs/apps/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/miniconda3/envs/apps/lib/python

In [53]:
es_retriever = ElasticsearchRetriever(es, index_name, k=10)
hybrid_retriever = HybridRetriever(es_retriever, vector_retriever, alpha=0.5)

In [54]:
query = resume_text
docs = hybrid_retriever.get_relevant_documents(query)

for i, doc in enumerate(docs, 1):
    print(f"[{i}] 공고 번호: {doc.metadata.get('rec_idx')} | 회사: {doc.metadata.get('company_nm')}")
    print(doc.page_content[:200] + "...\n")


/root/miniconda3/envs/apps/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/tmp/ipykernel_1901/2073118433.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  vec_docs = self.vector_retriever.get_relevant_documents(query)


[1] 공고 번호: 50226095 | 회사: (주)엘지씨엔에스
2025년 상반기 신입사원 채용 채용공고 상세
㈜LG CNS 2025년 상반기 신입사원 채용
솔루션·SI·ERP·CRM > SI·시스템통합, 시스템관리
기획·전략 > 직무·직업 > 웹기획
기획·전략 > 직무·직업 > 전략기획
기획·전략 > 직무·직업 > PL(프로젝트리더)
기획·전략 > 직무·직업 > PM(프로젝트매니저)
기획·전략 > 직무·직업 > PMO...

[2] 공고 번호: 50280660 | 회사: (주)팜스토리 서울사료 인천
(주)팜스토리서울사료 수의사 신입/경력 특별채용


사람인 
 
 
 
본문 바로가기
2025년 상반기 공채 시작!
2025년 상반기 공채 시작!
로그인

회원가입
기업서비스
로그인
회원가입

기업홈
공고 등록
지원자 관리
인재풀
스마트 리크루터
인적성 · 평가도구
HR매거진
채용상품
사람인 비즈니스
전체메뉴
채용정보
지역별
직업별
역세권별
HOT100
헤...

[3] 공고 번호: 49902739 | 회사: 블리비의원
[서울] 블리비의원 상담실장 신규/추가 모집


사람인 
 
 
 
본문 바로가기
2025년 상반기 공채 시작!
2025년 상반기 공채 시작!
로그인

회원가입
기업서비스
로그인
회원가입

기업홈
공고 등록
지원자 관리
인재풀
스마트 리크루터
인적성 · 평가도구
HR매거진
채용상품
사람인 비즈니스
전체메뉴
채용정보
지역별
직업별
역세권별
HOT100
헤드헌...

[4] 공고 번호: 50283279 | 회사: 이지스터디학원
대구시 동구  수학학원강사 신입&경력 채용 ( 초중등 강사)


사람인 
 
 
 
본문 바로가기
2025년 상반기 공채 시작!
2025년 상반기 공채 시작!
로그인

회원가입
기업서비스
로그인
회원가입

기업홈
공고 등록
지원자 관리
인재풀
스마트 리크루터
인적성 · 평가도구
HR매거진
채용상품
사람인 비즈니스
전체메뉴
채용정보
지역별
직업별
역세권별
H...

[5] 공고 번호: 50273596 | 회사: 설연고학원
업계최고